<a href="https://colab.research.google.com/github/unconst/GradientBidding/blob/master/Singlebidder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Bidding: Single Bidder

In [1]:
!wget https://github.com/unconst/GradientBidding/raw/master/utils_moe.py

--2020-01-23 02:04:21--  https://github.com/unconst/GradientBidding/raw/master/utils_moe.py
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/unconst/GradientBidding/master/utils_moe.py [following]
--2020-01-23 02:04:22--  https://raw.githubusercontent.com/unconst/GradientBidding/master/utils_moe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37803 (37K) [text/plain]
Saving to: ‘utils_moe.py’

utils_moe.py        100%[===================>]  36.92K  --.-KB/s    in 0.008s  

2020-01-23 02:04:22 (4.35 MB/s) - ‘utils_moe.py’ saved [37803/37803]



In [2]:
import tensorflow as tf
import types
from utils_moe import noisy_top_k_gating
from utils_moe import SparseDispatcher
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/d

In [0]:
# FFNN with biases.
def expert(i, x, hparams):
    with tf.compat.v1.variable_scope("expert"):
        sizes = [hparams.n_inputs] + [hparams.e_hidden for _ in range(hparams.e_layers)] + [hparams.n_embedding]
        for i in range(len(sizes) - 1):
            w = tf.Variable(tf.truncated_normal([sizes[i], sizes[i+1]], stddev=0.1))
            b = tf.Variable(tf.constant(0.1, shape=[sizes[i+1]]))
            x = tf.matmul(x, w) + b
    return x

# Cross entropy loss + accuracy.
def target_loss(embedding, targets, hparams):
    with tf.compat.v1.variable_scope("target_loss"):
        w = tf.Variable(tf.truncated_normal([hparams.n_embedding, hparams.n_targets], stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[hparams.n_targets])),
        logits = tf.add(tf.matmul(embedding, w), b)
        target_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=targets, logits=logits))
        correct = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        return target_loss, accuracy

In [0]:
# Incentive function inputs weights, outputs revenue.
# This the most basic, just takes the inloop weight as your score
def incentive_fn(weights, hparams):
    weights = tf.linalg.normalize(weights)
    return tf.slice(weights, [0], [1])

In [0]:
def model_fn(hparams):    
    x_inputs = tf.placeholder("float", [None, hparams.n_inputs], 'inputs')
    y_targets = tf.placeholder("float", [None, hparams.n_targets], 'targets')    
    
    # Sparsely gated mixture of experts with choice k. Produces an importance score 
    # for each x_input then chooses the topk. These children recieve the outgoing query.
    # expert_inputs is a list of tensors, inputs for each expert.
    gates, load = noisy_top_k_gating(x_inputs, hparams.n_experts, train = True, k = hparams.k)
    dispatcher = SparseDispatcher(hparams.n_experts, gates)
    expert_inputs = dispatcher.dispatch(x_inputs)

    # Basic importance scores can attained from the gating network by summing over the importance 
    # of each example. We choose a 'self-importance' score here which counts as the in-loop to our
    # incentive function. The network should try to maximize this value.
    importance = tf.linalg.normalize(tf.reduce_sum(gates, 0))[0]
    self_weight = tf.Variable(tf.constant([1.0]))
    weights = tf.linalg.normalize(tf.concat([self_weight, importance], axis=0))[0]
    revenue = tf.slice(weights, [0], [1])
    
    # Dispatch the inputs to the experts. We mask the responses with a faux-bidding system,
    # here, we set a mask w.r.t the bids with a hparams.market_shift shifted relu. Bids that
    # drop bellow the market shift should zero out.
    expert_outputs = []
    expert_masks = []
    for i in range(hparams.n_experts):
        expert_output = expert(i, expert_inputs[i], hparams)
        
        # Apply mask to the output.
        expert_mask = tf.nn.relu(tf.slice(weights, [i], [1]) - hparams.market_shift)
        masked_output = expert_mask * expert_output
        
        expert_masks.append(expert_mask)
        expert_outputs.append(masked_output)
    expert_masks = tf.concat(expert_masks, axis=0)

    
    # Combine the expert_inputs.
    embedding = dispatcher.combine(expert_outputs)
        
    # Loss and accuracy stuff.
    loss, accuracy = target_loss(embedding, y_targets, hparams)
    
    # Run the step: optimize for loss and revenue. 
    train_step = tf.train.AdamOptimizer(hparams.learning_rate).minimize(loss - revenue)
    
    metrics = {
        'loss': loss,
        'revenue': revenue,
        'accuracy': accuracy,
        'importance': importance,
        'weights': weights,
        'masks': expert_masks,
    }
    return train_step, metrics

In [7]:
hparams = types.SimpleNamespace( 
    n_inputs = 784,
    n_targets = 10,
    k = 3,
    n_experts = 3,
    e_layers = 2,
    e_hidden = 256,
    n_embedding = 256,
    batch_size=256,
    learning_rate=1e-3,
    n_iterations = 10000,
    n_print = 100,
    market_shift = 0.2,
)

graph = tf.Graph()
session = tf.Session(graph=graph)
with graph.as_default():
    train_step, metrics = model_fn(hparams)
    session.run(tf.global_variables_initializer())

for i in range(hparams.n_iterations):
    batch_x, batch_y = mnist.train.next_batch(hparams.batch_size)
    feeds = {'inputs:0': batch_x, 'targets:0': batch_y}
    session.run(train_step, feeds)

    if i % hparams.n_print == 0:
        feeds = {'inputs:0': batch_x, 'targets:0': batch_y}
        train_metrics = session.run(metrics, feeds)
        print("Iteration: " + str(i))
        for key in train_metrics:
            print (str(key) + ":  " + str(train_metrics[key]))
        print ('-')


Iteration: 0
loss:  1.8183292
revenue:  [0.7074601]
accuracy:  0.5234375
importance:  [0.5131693  0.63815343 0.57394904]
weights:  [0.7074601  0.3626841  0.45101705 0.4056404 ]
masks:  [0.5074601  0.1626841  0.25101703]
-
Iteration: 100
loss:  0.23909731
revenue:  [0.7403392]
accuracy:  0.9296875
importance:  [0.59748584 0.78650296 0.15628135]
weights:  [0.7403392  0.40165    0.52871364 0.10505757]
masks:  [0.54033923 0.20165001 0.32871366]
-
Iteration: 200
loss:  0.1776205
revenue:  [0.7665271]
accuracy:  0.9375
importance:  [0.57273656 0.7867943  0.2300598 ]
weights:  [0.7665271  0.36781827 0.5052887  0.14774716]
masks:  [0.5665271  0.16781826 0.30528873]
-
Iteration: 300
loss:  0.17145687
revenue:  [0.78780156]
accuracy:  0.95703125
importance:  [0.54288346 0.7867947  0.29365212]
weights:  [0.78780156 0.3343777  0.48460975 0.18086888]
masks:  [0.5878016  0.1343777  0.28460974]
-
Iteration: 400
loss:  0.1533003
revenue:  [0.80568117]
accuracy:  0.94921875
importance:  [0.62325436 0.7